# To do list

1. Mapas por equipos -> Vicente
2. Career jugador -> Sebastián
3. stats round pistols por equipos -> Aldo
4. stats ftu por equipos -> Aldo
5. Guardar los DataFrames en un CSV -> Aldo

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import cloudscraper
from datetime import date
from time import sleep

In [4]:
class HltvScraper():
    def __init__(self) -> None:
        self.scraper = cloudscraper.create_scraper(
            browser={"browser": "firefox", "platform": "windows"}
        )
        self.url_base = "https://www.hltv.org"

        today = date.today()
        self.def_params(today.replace(year=2023), today, "all", "all", "all")

    def def_params(self, statDate, endDate, matchType, maps, rankingFilter) -> None:
        """
        Define los paremetros de consulta.
        """

        self.statDate = statDate
        self.endDate = endDate
        self.matchType = matchType
        self.maps = maps
        self.rankingFilter = rankingFilter
        
        self.params = (f"?startDate={self.statDate}&endDate={self.endDate}"
                       f"&matchType={self.matchType}&maps={self.maps}"
                       f"&rankingFilter={self.rankingFilter}")
        
    def players_of_team(self, team: str) -> dict:
        """
        team: /id/nombre (e.g. /4608/natus-vincere)
        """

        url_team = self.url_base + "/stats/teams/" + team + self.params
        response = self.scraper.get(url_team)

        if response.status_code != 200:
            return

        soup = bs(response.text)
        list_reset_grid = soup.find_all(class_="grid reset-grid")
        list_team_info = list_reset_grid[0].find_all(class_="teammate-info standard-box")

        players = {}
        for box in list_team_info:
            link = box.find("a")["href"]
            teammate = box.find(class_="text-ellipsis").text

            players[teammate] = link

        return players
    
    def individual_stats(self, player: str) -> pd.DataFrame:
        """
        Retorna un DataFrame con las estadísticas individuales del jugador consultado

        player: /id/nickname (e.g. /9816/aleksib)
        """

        nickname = player.split("/")[2]
        url_team = self.url_base + "/stats/players/individual" + player + self.params
        response = self.scraper.get(url_team)

        if response.status_code != 200:
            return
        
        soup = bs(response.text)
        tabla = soup.find(class_="columns")
        list_standard_box = tabla.find_all(class_="standard-box")

        counter = 0
        stats = {}
        group = ["Overall stats", "Opening stats", "Round stats", "Weapon stats"]
        
        for box in list_standard_box:
            stat_group = {}
            
            for row in box.find_all(class_="stats-row"):
                labels = [span.text for span in row.find_all('span')]
                stat_group[labels[0]] = labels[-1]

            stats[group[counter]] = stat_group
            counter += 1

        # Gracias ChatGPT, código para crear un DataFrame con subcolumnas
        df = pd.DataFrame(
            {
                (outer_key, inner_key): value 
                   for outer_key, inner_dict in stats.items() 
                   for inner_key, value in inner_dict.items()
            },
            index=[nickname]
        )
        
        return df
    
    def stats_players_team(self, team: str) -> pd.DataFrame:
        """
        Retorna un DataFrame con las estadísticas individuales de cada integrante del
        equipo consultado.

        team: /id/nombre (e.g. /4608/natus-vincere)
        """

        df_players = pd.DataFrame()
        dict_players = self.players_of_team(team)

        counter = 0
        while not dict_players and counter < 10:
            sleep(1)
            print("Reintentando hacer request...")
            dict_jugadores = self.players_of_team(team)
            counter += 1

        if not dict_jugadores:
            return

        for link in dict_jugadores.values():
            player = link.split("?")[0][14:]
            df = self.individual_stats(player)
            df_players = pd.concat([df_players, df])
            
        return df_players
    
    def teams_major_qualifier(self) -> list:
        """
        Retorna una lista de diccionarios con los grupos de los equipos de las clasificatorias
        del major, con el formato "Nombre_equipo": "/id/nombre"
        """
        ulr_qualifier = "https://www.hltv.org/major/qualifier"
        response = self.scraper.get(ulr_qualifier)

        soup = bs(response.text)
        list_team_grid = soup.find_all(class_="majorTabSection teamsGridContainer")

        list_of_groups = []

        for team_grid in list_team_grid:
            list_hrefs = team_grid.find_all("a")
            group = {}
            for href in list_hrefs:
                id = href["href"][11:]
                team = href.find(class_="text-ellipsis").text
                group[team] = id
            
            list_of_groups.append(group)

        return list_of_groups
    
    def mapa_equipo(self, equipo, mapa) -> pd.DataFrame:
        pass

In [5]:
hltv_scraper = HltvScraper()

In [6]:
dicc = hltv_scraper.players_of_team("/4608/natus-vincere")
dicc

{'Aleksib': '/stats/players/9816/aleksib?startDate=2023-11-08&endDate=2024-11-08&matchType=all&maps=all&rankingFilter=all',
 'iM': '/stats/players/14759/im?startDate=2023-11-08&endDate=2024-11-08&matchType=all&maps=all&rankingFilter=all',
 'b1t': '/stats/players/18987/b1t?startDate=2023-11-08&endDate=2024-11-08&matchType=all&maps=all&rankingFilter=all',
 'jL': '/stats/players/19206/jl?startDate=2023-11-08&endDate=2024-11-08&matchType=all&maps=all&rankingFilter=all',
 'w0nderful': '/stats/players/20127/w0nderful?startDate=2023-11-08&endDate=2024-11-08&matchType=all&maps=all&rankingFilter=all'}

In [3]:
df1 = hltv_scraper.individual_stats("/9816/aleksib")
df1

Overall stats                                                     \
                Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib          1985   2407         0.82         0.53              1503   

                                      Opening stats                       \
        Kill - Death difference Total opening kills Total opening deaths   
aleksib                    -422                 287                  377   

                                                ...   Round stats  \
        Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib               0.76                0.88  ...           326   

                                                  Weapon stats               \
        3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib            67             6             1         1324           19   

                                              
        SMG kills Pistol kills Grenade Other  
aleksib       259          320      64    20  

[1 rows x 24 columns]

In [ ]:
df_jugadores_navi = hltv_scraper.stats_players_team("/4608/natus-vincere")
df_jugadores_navi

Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...
Reintentando hacer request...


Overall stats                                                     \
                  Kills Deaths Kill / Death Kill / Round Rounds with kills   
aleksib            1985   2407         0.82         0.53              1503   
im                 2611   2501         1.04         0.70              1761   
b1t                2718   2426         1.12         0.73              1846   
jl                 2670   2425         1.10         0.71              1792   
w0nderful          2701   2172         1.24         0.72              1849   

                                        Opening stats                       \
          Kill - Death difference Total opening kills Total opening deaths   
aleksib                      -422                 287                  377   
im                            110                 460                  434   
b1t                           292                 436                  359   
jl                            245                 412                  358   
w0nderful                     529                 386                  229   

                                                  ...   Round stats  \
          Opening kill ratio Opening kill rating  ... 2 kill rounds   
aleksib                 0.76                0.88  ...           326   
im                      1.06                1.09  ...           491   
b1t                     1.21                1.09  ...           509   
jl                      1.15                1.06  ...           484   
w0nderful               1.69                1.08  ...           413   

                                                    Weapon stats               \
          3 kill rounds 4 kill rounds 5 kill rounds  Rifle kills Sniper kills   
aleksib              67             6             1         1324           19   
im                  140            25             1         2087           13   
b1t                 142            25             1         2037           22   
jl                  154            26             2         2037           19   
w0nderful           168            29             4         1080         1100   

                                                
          SMG kills Pistol kills Grenade Other  
aleksib         259          320      64    20  
im               72          420      20    14  
b1t             198          440      23    12  
jl              142          441      27    25  
w0nderful        40          465      21    15  

[5 rows x 24 columns]

In [7]:
lista = hltv_scraper.teams_major_qualifier()

In [8]:
lista

[{'Natus Vincere': '/4608/Natus%20Vincere',
  'Vitality': '/9565/Vitality',
  'MOUZ': '/4494/MOUZ',
  'FaZe': '/6667/FaZe',
  'Falcons': '/11283/Falcons',
  'BetBoom': '/12394/BetBoom',
  'fnatic': '/4991/fnatic',
  'GamerLegion': '/9928/GamerLegion',
  'Sangal': '/10333/Sangal',
  'SINNERS': '/10577/SINNERS',
  'ECLOT': '/7441/ECLOT',
  'Nemiga': '/7969/Nemiga',
  'Cloud9': '/5752/Cloud9',
  'Rebels': '/12642/Rebels',
  'SAW': '/10567/SAW',
  'UNiTY': '/12267/UNiTY'},
 {'G2': '/5995/G2',
  'Spirit': '/7020/Spirit',
  'Eternal Fire': '/11251/Eternal%20Fire',
  'Virtus.pro': '/5378/Virtus.pro',
  'HEROIC': '/7175/HEROIC',
  'Astralis': '/6665/Astralis',
  '3DMAX': '/4914/3DMAX',
  'Ninjas in Pyjamas': '/4411/Ninjas%20in%20Pyjamas',
  'Aurora': '/11861/Aurora',
  'TSM': '/5996/TSM',
  'PARIVISION': '/12467/PARIVISION',
  'B8': '/11241/B8',
  'Passion UA': '/12426/Passion%20UA',
  '9 Pandas': '/11883/9%20Pandas',
  'BIG': '/7532/BIG',
  'Sashi': '/11712/Sashi'},
 {'Liquid': '/5973/Liquid'

In [ ]:
europa_1 = lista[0]
europa_2 = lista[1]
america_rmr = lista[2]
asia_rmr = lista[3]

asia_rmr

{'The MongolZ': '/6248/The%20MongolZ',
 'FlyQuest': '/12774/FlyQuest',
 'Lynn Vision': '/8840/Lynn%20Vision',
 'Rare Atom': '/11514/Rare%20Atom',
 'TALON': '/8210/TALON',
 'GR': '/12088/GR',
 'Alter Ego': '/9958/Alter%20Ego',
 'DRILLAS': '/12937/DRILLAS'}